# Graph Neural Networks - Practicals 1

Welcome to our hands-on practical guide to Graph Neural Networks (GNNs)! This tutorial is designed to provide you with a complete end-to-end experience in training GNNs and to expose you to the intricacies involved in the process.

Over the course of this practical session, you will:

- Experience running a complete end-to-end training of Graph Neural Networks.
- Implement two unique GNN layers, providing a deeper understanding of the internal operations of these networks.
- Be introduced to the innovative technique of Ordered Subgraph Aggregation, which we will explore in detail.

By the end of this session, you will have a solid understanding of GNNs and be equipped with the knowledge to experiment and build upon these foundational concepts.

Now, let's dive into it!

### [RUN] PyTorch Installation

Before we start, let's ensure that PyTorch, one of the key libraries we'll be using, is correctly installed. PyTorch is a popular open-source machine learning library and it's essential for training our GNNs.


In [1]:
import torch
torch.manual_seed(0)
TORCH = torch.__version__.split('+')[0]
DEVICE = 'cu' + torch.version.cuda.replace('.', '') if torch.cuda.is_available() else 'cpu'

!pip install -q torch-sparse torch-scatter torch-cluster torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{DEVICE}.html
!pip install -q torch-geometric
!pip install -q networkx==3.1 matplotlib tqdm torchmetrics ipywidgets

import torch.nn as nn
import torch.nn.functional as F
import torchmetrics
import torch_geometric.nn as gnn
import torch_geometric as tg
from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_networkx
import networkx as nx
import matplotlib.pyplot as plt
from typing import Tuple
from tqdm import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.1/504.1 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.3/732.3 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.7/205.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 23.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.8/728.8 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.0 MB/s eta 0:00:00


### [INFO 1] How are we training neural networks?


Recently, deep learning on graphs has emerged to one of the hottest research fields in the deep learning community.
Here, **Graph Neural Networks (GNNs)** aim to generalize classical deep learning concepts to irregular structured data (in contrast to images or texts) and to enable neural networks to reason about objects and their relations.

This is done by following a simple **neural message passing scheme**, where node features $\mathbf{x}_v^{(\ell)}$ of all nodes $v \in \mathcal{V}$ in a graph $\mathcal{G} = (\mathcal{V}, \mathcal{E})$ are iteratively updated by aggregating localized information from their neighbors $\mathcal{N}(v)$:

$$
\mathbf{x}_v^{(\ell + 1)} = f^{(\ell + 1)}_{\theta} \left( \mathbf{x}_v^{(\ell)}, \left\{ \mathbf{x}_w^{(\ell)} : w \in \mathcal{N}(v) \right\} \right)
$$

This tutorial will introduce you to some fundamental concepts regarding deep learning on graphs via Graph Neural Networks based on the **[PyTorch Geometric (PyG) library](https://github.com/rusty1s/pytorch_geometric)**.
PyTorch Geometric is an extension library to the popular deep learning framework [PyTorch](https://pytorch.org/), and consists of various methods and utilities to ease the implementation of Graph Neural Networks.


### Download Training Data

**Datasets**

When comparing Graph Neural Networks (GNNs) to other neural network architectures such as Fully-Connected Neural Networks (FCNNs) or Convolutional Neural Networks (CNNs), there are unique requirements for the data used in GNNs.

GNNs typically require two fundamental data points: a graph structure, which describes how nodes are connected, and a feature matrix, providing information associated with each node.

For the purpose of this tutorial, we aim to keep the focus on benchmarking and comparing various architectures. Therefore, we'll reduce the number of input features with `reduce_features` initially. This step will help us evaluate the efficiency and performance of our models in a more streamlined context.

In [2]:
from torch_geometric.datasets import TUDataset

def reduce_features(data):
    data.x = data.x.sum(dim=1, keepdim=True)
    return data

dataset = TUDataset(root='data/TUDataset', name='MUTAG', pre_transform=reduce_features)

Extracting data/TUDataset/MUTAG/MUTAG.zip
Processing...
Done!


### [INFO 2] Explore dataset

The most common task for graph classification is **molecular property prediction**, in which molecules are represented as graphs, and the task may be to infer whether a molecule inhibits HIV virus replication or not.

The TU Dortmund University has collected a wide range of different graph classification datasets, known as the [**TUDatasets**](https://chrsmrrs.github.io/datasets/), which are also accessible via [`torch_geometric.datasets.TUDataset`](https://pytorch-geometric.readthedocs.io/en/latest/modules/datasets.html#torch_geometric.datasets.TUDataset) in PyTorch Geometric.
Let's load and inspect one of the smaller ones, the **MUTAG dataset**:

In [3]:
from ipywidgets import interact, IntSlider

@interact(index=IntSlider(max=len(dataset)-1))
def display_dataset_element(index):
    g = to_networkx(dataset[index], to_undirected=True)
    plt.title(f"Class: {dataset[index].y.item()}")
    nx.draw(g, with_labels=True)
    plt.show()

interactive(children=(IntSlider(value=0, description='index', max=187), Output()), _dom_classes=('widget-inter…

## [INFO 3] Mini-batching of graphs

Since graphs in graph classification datasets are usually small, a good idea is to **batch the graphs** before inputting them into a Graph Neural Network to guarantee full GPU utilization.
In the image or language domain, this procedure is typically achieved by **rescaling** or **padding** each example into a set of equally-sized shapes, and examples are then grouped in an additional dimension.
The length of this dimension is then equal to the number of examples grouped in a mini-batch and is typically referred to as the `batch_size`.

However, for GNNs the two approaches described above are either not feasible or may result in a lot of unnecessary memory consumption.
Therefore, PyTorch Geometric opts for another approach to achieve parallelization across a number of examples. Here, adjacency matrices are stacked in a diagonal fashion (creating a giant graph that holds multiple isolated subgraphs), and node and target features are simply concatenated in the node dimension:

This procedure has some crucial advantages over other batching procedures:

1. GNN operators that rely on a message passing scheme do not need to be modified since messages are not exchanged between two nodes that belong to different graphs.

2. There is no computational or memory overhead since adjacency matrices are saved in a sparse fashion holding only non-zero entries, *i.e.*, the edges.

PyTorch Geometric automatically takes care of **batching multiple graphs into a single giant graph** with the help of the [`torch_geometric.data.DataLoader`](https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html#torch_geometric.data.DataLoader) class.

## [INFO 4] Exploring Graph Classification with Graph Neural Networks

In this tutorial session, we will delve into the application of **Graph Neural Networks (GNNs) for graph classification**. Unlike node classification, graph classification is tasked with classifying entire graphs, given a **dataset of graphs**, based on their structural properties. Our aim is to generate embeddings for entire graphs in such a way that they are linearly separable for a given task.

A key part of this process is the training and evaluation of our model. We will use the `train_and_eval` function for this purpose:


In [4]:
def train_and_eval(model : nn.Module, dataset : Dataset, hparams: dict = {}) -> Tuple[nn.Module, float]:
    """
    Train and evaluate the model on the dataset.
    A boilerplate for training and evaluating a neural network.

    Args:
        model (nn.Module): The model to train and evaluate.
        dataset (Dataset): The dataset to train and evaluate on.
        hparams (dict): A dictionary of hyperparameters.

    Returns:
        nn.Module: The trained model with the best validation accuracy.
        float: The best validation accuracy.
    """

    hparams["num_epochs"] = hparams.get("num_epochs", 100)
    hparams["lr"] = hparams.get("lr", 0.01)

    # shuffle dataset
    dataset = dataset.shuffle()
    train_sizes = {"ENZYMES": 540, "MUTAG": 150}
    sz = train_sizes[dataset.name]

    # init training and validation dataloaders
    dataloader_train = DataLoader(dataset[:sz], batch_size=16, shuffle=True)
    dataloader_val = DataLoader(dataset[sz:], batch_size=16)

    # init optimizer and loss
    optimizer = torch.optim.Adam(model.parameters(), lr=hparams["lr"])
    criterion = nn.CrossEntropyLoss()

    # init metrics
    metric_loss_train = torchmetrics.MeanMetric()
    metric_accuracy_train = torchmetrics.Accuracy(task="multiclass", num_classes=dataset.num_classes)
    metric_accuracy_val = torchmetrics.Accuracy(task="multiclass", num_classes=dataset.num_classes)

    best_model_state = None
    best_val_acc = 0.0

    # training loop
    num_epochs = hparams["num_epochs"]
    epoch_bar = tqdm(list(range(num_epochs)), desc="Epochs" + " "*100)
    for epoch in epoch_bar:

        # training loop
        model.train()
        for data in dataloader_train:
            optimizer.zero_grad()
            logits = model(data.x, data.edge_index, data.batch)
            loss = criterion(logits, data.y)
            loss.backward()
            optimizer.step()

            metric_loss_train(loss)
            metric_accuracy_train(logits.argmax(dim=1), data.y)


        # evaluation loop
        model.eval()
        for data in dataloader_val:
            with torch.no_grad():
                logits = model(data.x, data.edge_index, data.batch)
                pred = logits.argmax(dim=1)
                metric_accuracy_val(pred, data.y)

        train_loss = metric_loss_train.compute().item()
        train_acc = metric_accuracy_train.compute().item()
        val_acc = metric_accuracy_val.compute().item()

        # progress bar
        epoch_bar.set_description("".join([
            f'Epoch: {epoch+1:03d} | ',
            f'Val Acc: {val_acc:.3f} | ',
            f'Train Acc: {train_acc:.3f} | ',
            f'Train Loss: {train_loss:.3f} | ',
            ]))

        # update best model
        if val_acc > best_val_acc:
            best_model_state = model.state_dict()
            best_val_acc = val_acc

    best_model = model
    best_model.load_state_dict(best_model_state)

    return best_model, best_val_acc

### [INFO 5] Define our model

We will use the `nn.Module` class from PyTorch to define our model.
Since having multiple datapoints we need to define a model that has multiple inputs in the `forward` function.

Training a GNN for graph classification usually follows a simple recipe:

1. Embed each node by performing multiple rounds of message passing
2. Aggregate node embeddings into a unified graph embedding (**readout layer**)
3. Train a final classifier on the graph embedding

There exists multiple **readout layers** in literature, but the most common one is to simply take the average of node embeddings:

$$
\mathbf{x}_{\mathcal{G}} = \frac{1}{|\mathcal{V}|} \sum_{v \in \mathcal{V}} \mathcal{x}^{(L)}_v
$$

PyTorch Geometric provides this functionality via [`gnn.global_mean_pool`](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.glob.global_mean_pool), which takes in the node embeddings of all nodes in the mini-batch and the assignment vector `batch` to compute a graph embedding of size `[batch_size, hidden_channels]` for each graph in the batch.

The final architecture for applying GNNs to the task of graph classification then looks as follows and allows for complete end-to-end training:

In [5]:
class SetConv(nn.Linear):
    """
    A simple linear layer that performs action on every node feature
    """

    def forward(self, x, edge_index):
        return super().forward(x)


class GraphNetModel(nn.Module):

    def __init__(self, num_features, num_classes, conv_layer_cls=None):
        super().__init__()

        # in case no gnnLayer is provided, use a simple linear layer
        if conv_layer_cls is None:
            conv_layer_cls = SetConv

        # first graph covolutional layer
        self.conv1 = conv_layer_cls(num_features, 32)
        self.conv2 = conv_layer_cls(32, 32)
        self.conv3 = conv_layer_cls(32, 32)
        # TODO🚀(optional):
        # - add more conv layers

        # last graph convolutional layer
        self.convL = conv_layer_cls(32, 32)

        # TODO🚀(optional):
        # - in many tutorials, the stantard dropout layer
        #    on **features** is used which can help
        #    with overfitting. Simlarly, the standard
        #    batch normalization can be used as well.
        # - `self.dropout = nn.Dropout(0.5)`

        # TODO🚀(optional):
        # - you might try to add batchnorm on features
        #   between the conv layers
        # - `self.batchnorm = nn.BatchNorm1d(32)`

        self.linear = nn.Linear(32, num_classes)

    def forward(self, x, edge_index, batch):

        # apply first layer and follow with ReLU
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
        x = self.conv3(x, edge_index).relu()

        # TODO🚀(optional):
        # - apply more conv layers

        x = self.convL(x, edge_index).relu()

        # this (set) pooling makes the function
        # invariant to the order of the nodes
        x = gnn.global_mean_pool(x, batch)

        # TODO🚀(optional):
        # add dropout or batchnorm of features
        # - `x = self.dropout(x)`

        # apply linear layer
        x = self.linear(x)

        return x

### [RUN] Initial Training and Evaluation without Incorporating Graph Convolutional Neural Network Layer

Let's begin our experiment with a simple run. This initial training and evaluation process does not involve any Graph Convolutional Neural Network layers. It allows us to gauge baseline performance and understand the effect of these layers when we introduce them later in the process.


In [6]:
model = GraphNetModel(dataset.num_features, dataset.num_classes)
model, val_accuracy = train_and_eval(model, dataset)

print("Model size:", sum(p.numel() for p in model.parameters()))
print("Best Accuracy:", val_accuracy)

Epoch: 100 | Val Acc: 0.632 | Train Acc: 0.673 | Train Loss: 0.632 | : 100%|██████████| 100/100 [00:13<00:00,  7.36it/s]

Model size: 3298
Best Accuracy: 0.6315789222717285


### [Task 1] Implementation of the GraphConv Layer

In this task, we will implement a Graph Convolutional (GraphConv) Layer. This layer is an integral part of Graph Neural Networks (GNNs) and allows for the propagation of information through the nodes of a graph.

The equation that governs the operation of the GraphConv layer is:

$$
\begin{align}
F^{(t+1)} &= \sigma(F^{(t)} W_1^{(t)} + A F^{(t)} W_2^{(t)} + b^{(t)})
\end{align}
$$

where,

- $F^{(t+1)}$ represents the features of the nodes in the graph at the next step (t+1).
- $F^{(t)}$ represents the features of the nodes in the graph at the current step (t).
- $W_1^{(t)}$ and $W_2^{(t)}$ are learnable weights at the current step (t).
- $A$ is the adjacency matrix representing the connections between nodes in the graph.
- $\sigma$ is the activation function (e.g., ReLU, sigmoid, tanh, etc.).
- $b^{(t)}$ is the learnable bias at the current step (t).

In this equation, we are updating the features of each node based on its own features and the features of its neighbors. The learnable weights and bias allow the model to adjust and optimize the impact of each node's features and its neighbors' features in the propagation process.

Your task is to implement this functionality in a GraphConv layer.


In [7]:
class OurGraphConvLayer(nn.Module):

    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.linear1 = nn.Linear(in_channels, out_channels)
        self.linear2 = nn.Linear(in_channels, out_channels, bias=False)

    def forward(self, x, edge_index):
        nx = x.shape[0]

        # create abstraction of sparse adjacency matrix from edge_index
        A = torch.sparse_coo_tensor(edge_index, torch.ones_like(edge_index[0]).float(), size=(nx, nx))


        W1 = self.linear1.weight.T
        W2 = self.linear2.weight.T
        b = self.linear1.bias

        out = x @ W1 + A @ x @ W2 + b

        return out

**test your model with GraphConv Layer:**
- This layer is also implemented in PyG as [torch_geometric.nn.GraphConv](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.GCNConv.html#torch_geometric.nn.conv.GraphConv).


In [8]:
model_our1 = GraphNetModel(dataset.num_features, dataset.num_classes, conv_layer_cls=OurGraphConvLayer)
trained_model_our1, val_accuracy_our1 = train_and_eval(model_our1, dataset)

print("Model size:", sum(p.numel() for p in model_our1.parameters()))
print("Best Accuracy:", val_accuracy_our1)

Epoch: 067 | Val Acc: 0.822 | Train Acc: 0.829 | Train Loss: 0.359 | :  67%|██████▋   | 67/100 [00:13<00:06,  4.89it/s]


KeyboardInterrupt: ignored

### [Task 2] Implementation of the Simplified Graph Convolutional Layer (GCN)

In this task, we delve deeper into the mechanics of Graph Neural Networks (GNNs) by implementing a simplified version of the Graph Convolutional Layer (GCN).

The operation of this simplified GCN is determined by the following equations:

$$
\begin{align}
\hat{D} &= D + I \\
\hat{A} &= A + I \\
\tilde{A} &= \hat{D}^{-1/2} \hat{A} \hat{D}^{-1/2} \\
F^{(t+1)} &= \sigma(\tilde{A} F^{(t)} W^{(t)} + b^{(t)})
\end{align}
$$

where,

- $\hat{D}$ and $\hat{A}$ are the degree matrix and adjacency matrix, respectively, with added self-connections (denoted by the identity matrix $I$). This modification ensures that each node receives its own features in addition to its neighbors'.
- $\tilde{A}$ is the normalized adjacency matrix. This normalization step makes the propagation of features across the graph more stable.
- $F^{(t+1)}$ represents the features of the nodes in the graph at the next step (t+1).
- $F^{(t)}$ represents the features of the nodes in the graph at the current step (t).
- $W^{(t)}$ is the learnable weight matrix at the current step (t).
- $\sigma$ is the activation function (e.g., ReLU, sigmoid, tanh, etc.).
- $b^{(t)}$ is the learnable bias at the current step (t).

The goal of this task is to implement this simplified GCN layer. This layer will help propagate and transform features across the graph, making it possible for our model to learn from the structure and features of the graph.


In [ ]:
class OurGCNConvLayer(nn.Module):

    def __init__(self, in_channels, out_channels):
        super().__init__()

        # initialize linear layer with weight `W` and bias `b`
        self.linear = nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        nx = x.shape[0]

        # create abstraction of sparse adjacency matrix from edge_index
        A = tg.utils.to_torch_coo_tensor(edge_index)

        # also sparse identity matrix
        I = torch.sparse_coo_tensor(torch.stack([torch.arange(nx), torch.arange(nx)]), torch.ones(nx).float(), size=(nx, nx))

        # add self-loops
        A_hat = A + I

        # degree vector
        deg_inv_sqrt = A_hat.sum(dim=1).to_dense().pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0.0

        # normalized inverse degree matrix
        Di = I * deg_inv_sqrt
        assert Di.is_sparse

        # using bias and weight of linear layer
        W = self.linear.weight.T
        b = self.linear.bias

        # normalize adjacency matrix
        A_norm = Di @ A_hat @ Di
        assert A_norm.is_sparse

        x = A_norm @ x @ W + b
        return x


**test your model with GCNConv Layer:**
 - note that this layer has less parameters that GraphConv and also does not differentiate between values of node's neightbors and value of node itself (!). This is very important to keep in mind for practical usage -- GCNConv does not work nicely with complente graphs or unlabeled graphs (our case).
 - Even though, you might find this layer useful in the following Task 3.
 - This layer is also imple
 mented in PyG as [torch_geometric.nn.GCNConv](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.GCNConv.html#torch_geometric.nn.conv.GCNConv).

In [ ]:
model_our2 = GraphNetModel(dataset.num_features, dataset.num_classes, conv_layer_cls=OurGCNConvLayer)
trained_model_our2, val_accuracy_our2 = train_and_eval(model_our2, dataset)

print("Model size:", sum(p.numel() for p in model_our2.parameters()))
print("Best Accuracy:", val_accuracy_our2)

### But what structures are our models really capturing?

Consider the following graphs:

In [ ]:
graph_G = nx.from_edgelist([(0,1), (1,2), (2,0), (3,4), (4,5), (5,3), (1,4)])
graph_H = nx.from_edgelist([(0,1), (1,2), (2,3), (3,4), (4,5), (5,0), (1,4)])

plt.subplots(1,2, figsize=(10,5))
plt.subplot(121)
plt.title("G")
nx.draw_networkx(graph_G)
plt.subplot(122)
plt.title("H")
nx.draw_networkx(graph_H)
plt.axis('off')
plt.show()

### So why are outputs identical if the graphs are clearly not isomorphic?
You might be wondering, why do we encounter identical outputs when the graphs are clearly non-isomorphic? The answer lies in the 1-Weisfeiler-Lehman test, commonly referred to as 1-WL or [Color Refinement](https://en.wikipedia.org/wiki/Colour_refinement_algorithm).



In [ ]:
model_our1.eval()

def from_networkx(G: nx.Graph) -> tg.data.Data:
    """Utility function"""
    data = tg.utils.from_networkx(G)
    data.x = torch.ones((G.number_of_nodes(), 1), dtype=torch.float)
    return data

# simple features to test the model
data_G = from_networkx(graph_G)
data_H = from_networkx(graph_H)
# only one batch
batch = torch.zeros(6).long()

logits_G = model_our1(data_G.x, data_G.edge_index, batch)
logits_H = model_our1(data_H.x, data_H.edge_index, batch)

print("Prediction for G: ", logits_G.detach().numpy())
print("Prediction for H:", logits_H.detach().numpy())

Breaking the Symmetry
However, even in such a situation, we are not without options. We can attempt to break this symmetry through permutation-invariant operations such as node marking or node deletion. It is clear that certain sets of graphs do not even share the same degree sequences, highlighting their distinctness. This approach could potentially allow us to distinguish between non-isomorphic graphs that were previously yielding identical outputs. We will cover this in the following section

# Ordered Subgraph Aggregation




The `graph_into_subgraphs` function is a key component in our preprocessing stage. It is responsible for transforming the original graph into subgraphs. This is achieved by duplicating each node representation for every other node and concatenating an identity matrix to these duplicates, which essentially allows us to embed the node's local context. Additionally, it manipulates the edge indices in such a way that edges are created between corresponding nodes across the subgraphs

In [ ]:
def graph_into_subgraphs(data):
    # 'n' captures the number of nodes in the graph
    # 'm' captures the number of edges in the graph
    n, m = len(data.x), len(data.edge_index[0])

    # Duplicate the node features for each node in the graph, and adds unique
    # identifier on each node, allowing us to capture the local context
    data.x = torch.cat([data.x.repeat(n, 1), torch.eye(n).view(-1, 1)], dim=1)

    # Adjust the edge indices to create new edges in the subgraphs
    # This is done by repeating the edge_index tensor 'n' times,
    # and adding an offset for each node's corresponding edges
    data.edge_index = data.edge_index.repeat(1, n) + n*torch.arange(n).repeat_interleave(m)

    return data

def draw_graph_with_aligned_components(graph, name):
    pos = nx.spring_layout(graph, seed=42)
    n = graph.number_of_nodes()
    data = from_networkx(graph)
    data.x = torch.ones(len(graph), 1)
    bagged_data = graph_into_subgraphs(data)
    bagged_graph = to_networkx(bagged_data, to_undirected=True)
    plt.subplots(1,6, figsize=(3*n,3))
    plt.suptitle(name, fontsize=16)
    for i, conn_comp in enumerate(nx.connected_components(bagged_graph)):
        plt.subplot(1,6,i+1)
        conn_g = bagged_graph.subgraph(conn_comp)
        nodes = sorted(conn_comp)
        pos_updated = {k: pos[j] for j, k in enumerate(nodes)}
        marks = bagged_data.x[torch.LongTensor(nodes), -1]
        nx.draw_networkx(conn_g, with_labels=True, node_color=marks, font_color='gray', pos=pos_updated)
        plt.axis('off')
    plt.show()



draw_graph_with_aligned_components(graph_G, "graph_into_subgraphs(G)")
draw_graph_with_aligned_components(graph_H, "graph_into_subgraphs(H)")


### Updating dataset

We are applying a transformation step on the TUDataset by using the function `graph_into_subgraphs`. The choice of this transformation function is inspired by the approach presented in the research paper titled [Ordered Subgraph Aggregation Networks](https://arxiv.org/pdf/2206.11168.pdf).

This function allows us to decompose each graph into subgraphs, capturing the local structural information around each node. When training Graph Neural Networks (GNNs), this local information can be aggregated to generate more meaningful node representations that account for both local and global structural information.

By applying this transformation at the dataset level, as `transform=graph_into_subgraphs`, we can efficiently preprocess our data. This is an advantageous approach, as opposed to implementing the same operation as part of the GNN model, which might introduce computational overhead during the training process.



In [ ]:
# Open the dataset with `transform=graph_into_subgraphs`
bagged_dataset = TUDataset(root='data/TUDataset', name='MUTAG', pre_transform=reduce_features, transform=graph_into_subgraphs)

In [ ]:
model_our_bagged = GraphNetModel(bagged_dataset.num_features, bagged_dataset.num_classes, OurGCNConvLayer)
trained_model_our_bagged, val_accuracy_our_bagged = train_and_eval(model_our_bagged, bagged_dataset)

print("Model size:", sum(p.numel() for p in model_our_bagged.parameters()))
print("Best Accuracy:", val_accuracy_our_bagged)

**With our modifications, our model has demonstrated improvements not only in terms of accuracy but also in its expressive power:**

In [ ]:
model_our_bagged.eval()

# simple features to test the model
data_G = graph_into_subgraphs(from_networkx(graph_G))
data_H = graph_into_subgraphs(from_networkx(graph_H))
batch = torch.zeros(len(data_G.x)).long()

logits_G = model_our_bagged(data_G.x, data_G.edge_index, batch)
logits_H = model_our_bagged(data_H.x, data_H.edge_index, batch)

print("Prediction for G: ", logits_G.detach().numpy())
print("Prediction for H:", logits_H.detach().numpy())

## Enhanced Expressivity!
This term refers to our model's improved ability to capture and articulate intricate patterns within our data. Through the use of Ordered Subgraph Aggregation, our model can now distinguish between more complex and nuanced graph structures. This elevates the sophistication of our predictions and enhances our model's overall performance.

## [TASK 3] (Optional) Time to Optimize your Graph Neural Network!
Now, it's time to head back to the `GraphNetModel` and `train_and_eval` functions. Seize this opportunity to devise a more efficient model on the dataset, or more likely, the bagged_dataset.

This step is optional but highly encouraged. By refining and optimizing your model, you not only enhance its performance but also deepen your understanding of the inner workings of Graph Neural Networks. So go ahead, push your boundaries and let's see how much your model can achieve!

[*] Text sections INFO1-INFO5 were mostly adapted from Pytorch Documentation, please visit [PyG Colab Tutorial](https://pytorch-geometric.readthedocs.io/en/latest/get_started/colabs.html) for more information.
